In [ ]:
#Q1
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import re

# 評分規則：符合一項得 1 分，滿分 5 分
RULES = {
    "length ≥ 8":        lambda s: len(s) >= 8,
    "uppercase letter":  lambda s: bool(re.search(r"[A-Z]", s)),
    "lowercase letter":  lambda s: bool(re.search(r"[a-z]", s)),
    "digit":             lambda s: bool(re.search(r"\d", s)),
    "special character": lambda s: bool(re.search(r"[^\w\s]", s)),  # 非英數與底線
}

def strength_score(password: str) -> tuple[int, list[str]]:
    """
    傳回 (分數, 未通過規則清單)
    """
    failed = [rule for rule, check in RULES.items() if not check(password)]
    return 5 - len(failed), failed

def main() -> None:
    try:
        pwd = input("請輸入欲檢測的密碼：")
    except KeyboardInterrupt:
        print("\n中斷輸入，程式結束。")
        return

    score, failed_rules = strength_score(pwd)
    print(f"密碼：{pwd}")
    print(f"強度：{score}/5")



if __name__ == "__main__":
    main()


密碼：
強度：0/5


IndentationError: expected an indented block after 'if' statement on line 11 (279770915.py, line 12)

In [6]:
#Q2
import cv2
import numpy as np
import os

# ----------------------
# 1. 輸入圖片路徑
# ----------------------
img_path = input("請輸入圖片路徑（例如 ./input.jpg）：").strip()

if not os.path.isfile(img_path):
    raise IOError(f"❌ 找不到檔案：{img_path}")

img = cv2.imread(img_path)
if img is None:
    raise IOError("❌ 無法讀取圖片，請確認檔案是否為圖片格式")

base_name = os.path.splitext(os.path.basename(img_path))[0]  # e.g. "input"

# ----------------------
# 2. 轉灰階
# ----------------------
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imwrite(f'{base_name}_gray.jpg', gray)

# ----------------------
# 3. Canny 邊緣偵測
# ----------------------
edges = cv2.Canny(gray, threshold1=100, threshold2=200)
cv2.imwrite(f'{base_name}_edges.jpg', edges)

# ----------------------
# 4. 旋轉 + 縮放
# ----------------------
def rotate_and_scale(image, angle, scale):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, scale)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
    M[0, 2] += (nW / 2) - center[0]
    M[1, 2] += (nH / 2) - center[1]
    return cv2.warpAffine(image, M, (nW, nH))

rotated_scaled = rotate_and_scale(img, angle=30, scale=0.6)
cv2.imwrite(f'{base_name}_rotate_scale.jpg', rotated_scaled)

# ----------------------
# 5. 指定顏色（藍）分割
# ----------------------
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
lower_blue = np.array([100, 150, 50])
upper_blue = np.array([140, 255, 255])
mask = cv2.inRange(hsv, lower_blue, upper_blue)
result = cv2.bitwise_and(img, img, mask=mask)

cv2.imwrite(f'{base_name}_blue_only.png', result)
cv2.imwrite(f'{base_name}_mask.png', mask)

print("✅ 全部處理完成！輸出檔案與原圖同一資料夾。")


OSError: ❌ 無法讀取圖片，請確認檔案是否為圖片格式

In [ ]:
#Q3
#!/usr/bin/env python3
import hashlib, secrets, string, sys

def hash_string(msg: str) -> str:
    """回傳完整 64 字元（256 bits）SHA-256 十六進位字串"""
    return hashlib.sha256(msg.encode('utf-8')).hexdigest()

def random_string(length: int = 16) -> str:
    """產生隨機字串（英數）"""
    alphabet = string.ascii_letters + string.digits
    return ''.join(secrets.choice(alphabet) for _ in range(length))

def truncate(hex_digest: str, bits: int) -> str:
    """將十六進位雜湊值『截斷』到前 bits 位元"""
    if bits <= 0:                   # 不截斷
        return hex_digest
    if bits > 256:
        raise ValueError("bits 需介於 0~256")
    # 1 hex = 4 bits
    hex_len = (bits + 3) // 4       # ceiling
    truncated = hex_digest[:hex_len]
    # 若 bits 不是 4 的倍數，把最後 4bit 再切半
    extra = (hex_len * 4) - bits
    if extra:
        truncated = truncated[:-1] + format(int(truncated[-1], 16) >> extra, 'x')
    return truncated

def find_collision(bits: int = 20):
    """回傳 (hash值, 字串1, 字串2, 嘗試次數)"""
    seen = {}               # truncated_hash -> original string
    tries = 0
    while True:
        tries += 1
        s = random_string()
        full = hash_string(s)
        t = truncate(full, bits)
        if t in seen and seen[t] != s:
            return t, seen[t], s, tries
        seen[t] = s

def main():
    print("=== SHA-256 雜湊 & 碰撞模擬系統 ===")
    while True:
        print("\n1) 查詢字串 SHA-256")
        print("2) 進行碰撞實驗")
        print("3) 離開")
        choice = input("請輸入選項: ")
        if choice == '1':
            msg = input("輸入字串: ")
            print("SHA-256 =", hash_string(msg))
        elif choice == '2':
            bits = int(input("要比對前幾個 bits？(建議 16~32, 上限 256): "))
            print(f"開始尋找 {bits} bits 碰撞…(可能需要等待)")
            h, s1, s2, n = find_collision(bits)
            print(">>> 找到碰撞！😎🎉")
            print(f"   截斷雜湊值 = {h}")
            print(f"   字串 1 = {s1}")
            print(f"   字串 2 = {s2}")
            print(f"   嘗試次數 = {n}")
            # 理論期望 ~ 2^(bits/2) 次
        elif choice == '3':
            print("Bye~ 👋")
            sys.exit(0)
        else:
            print("無效選項，請重來～")

if __name__ == "__main__":
    main()